# Visualization

Test ClusterWay to automatically detect waypoints from a occupancy grid map of a row based crop and cluster them.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import tensorflow as tf

from utils.tools import load_config, deepWayLoss, clusterLoss
from utils.models import build_deepway, build_clusterway
from utils.dataset import load_dataset_test
from utils.train import Trainer
from utils.visualization import RotationalPredictor

In [ ]:
# select a GPU and set memory growth 
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
# important paths and names
PATH_DIR = os.path.abspath('.')
PATH_WEIGHTS = os.path.join(PATH_DIR, 'bin')

TEST_DATA_PATH = os.path.join(PATH_DIR, '../Datasets/straight/test')
TEST_CURVED_DATA_PATH = os.path.join(PATH_DIR, '../Datasets/curved/test') # curved
SATELLITE_DATA_PATH = os.path.join(PATH_DIR, '../Datasets/satellite/')
SATELLITE_CURVED_DATA_PATH = os.path.join(PATH_DIR, '../Datasets/satellite_curved') # curved

config_file = 'config.json'
config = load_config(config_file)

In [ ]:
#select model

#name_model = 'deep_way_pretrained'
name_model = 'cluster_way_pretrained'
CURVED = False
I = 0

name_model += '_curved' if CURVED else ''
name_model += f'_{I}'
name_model

# 1 Import the Test Dataset

In [ ]:
#choose target dataset
img_folder = SATELLITE_DATA_PATH

X_test, y_test, y_cluster_test, df_waypoints_test = load_dataset_test(img_folder, config)
print(X_test.shape, y_test.shape, y_cluster_test.shape)

# 2 Create the model

In [ ]:
# create model
tf.keras.backend.clear_session()
if 'deep_way' in name_model:
    deepway_net = build_deepway(name_model, config['FILTERS'],
                            config['KERNEL_SIZE'],
                            config['R'], config['MASK_DIM'])
elif 'cluster_way' in name_model:
    j = name_model.find('cluster_way')
    name_classic = name_model[:j] + 'deep_way' + name_model[j+11:]
    model_classic = build_deepway(name_classic, config['FILTERS'],
                            config['KERNEL_SIZE'],
                            config['R'], config['MASK_DIM'], True)

    deepway_net = build_clusterway(name_model, model_classic, config['FILTERS'],
                            config['KERNEL_SIZE'], out_feats=config['OUT_FEATS'])
else:
    raise ValueError(f'Wrong model {name_model}.')

In [ ]:
deepway_net.summary()

In [ ]:
# load weights
loss={'mask': deepWayLoss('none')}
if 'cluster_way' in name_model:
    loss['features'] = clusterLoss('none')
trainer = Trainer(deepway_net, config, loss=loss, optimizer=tf.keras.optimizers.Adam(0.), checkpoint_dir=PATH_WEIGHTS)

# 3 Visualize predictions

In [ ]:
predictor = RotationalPredictor(deepway_net, X_test, config)
predictor.start()